In [1]:
import pypsa
import pandas as pd
import matplotlib.pyplot as plt
from bc_power import utils

In [53]:
config_file = r"/home/pmcwhannel/repos/PyPSA_BC/config/config.yaml"
cfg = utils.load_config(config_file)
network_path = "/home/pmcwhannel/repos/PyPSA_BC/results/network_debug.nc" # cfg['pypsa']['results']

n = pypsa.Network(override_component_attrs=utils.get_multi_link_override())
pypsa.Network.import_from_netcdf(network=n,path=network_path)

INFO:pypsa.io:Imported network network_debug.nc has buses, carriers, generators, lines, line_types, links, loads, stores, transformer_types


### Annual Energy check

In [269]:
def calculate_vre_energy(p_max_pu, p_nom):
    total = 0
    for name,p in p_nom.items():
        total += sum(p_max_pu[name] * p)

    return total


In [283]:
# Calculate total potential energy from hydro in system and total demand at annual level to start.
# Load
total_load = n.loads_t.p_set.sum().sum() # MW-hr for the year

# Generators:
# Total RoR Generation ()
ror_cols = [col for col in n.generators.index if 'RoR Generator' in col]
total_ror = calculate_vre_energy(n.generators_t.p_max_pu.loc[:,ror_cols], n.generators.loc[ror_cols,:].p_nom)

# Total Wind Generation ()
ror_cols = [col for col in n.generators.index if 'Wind Generator' in col]
total_wind = calculate_vre_energy(n.generators_t.p_max_pu.loc[:,ror_cols], n.generators.loc[ror_cols,:].p_nom)

# Total Wind Generation ()
ror_cols = [col for col in n.generators.index if 'Solar Generator' in col]
total_pv = calculate_vre_energy(n.generators_t.p_max_pu.loc[:,ror_cols], n.generators.loc[ror_cols,:].p_nom)


In [271]:
def STC_check(name):
    if "STC" in name:
        return True
    else:
        return False
    
def check_ror_water(name):
    if "BC_WDN_RES" in name:
        return True
    if "BC_WHN_RES" in name:
        return True
    return False


# 1) Find an Inflow Generator
# Start with an Inflow Generator 
inflow_generators = [ig for ig in n.generators.index if 'Inflow Generator' in ig] #.remove(['BC_WDN_RES Inflow Generator','BC_WHN_RES Inflow Generator'])
inflow_generators.remove('BC_WDN_RES Inflow Generator')
inflow_generators.remove('BC_WHN_RES Inflow Generator')
total_energy = 0

for ig in inflow_generators:#inflow_generators:
    water_bus = ig.split()[0] + " Water Bus"
    
    if STC_check(water_bus): # Special case for STC since removal
        continue

    discharge_link = n.links[n.links.bus0 == water_bus].index[0].split()[0] + " Discharge Link"
    
    if sum(n.generators_t.p_set.columns == ig) == 0:
        # There is no inflow generator
        # This will produce 0 energy for all assets downstream
        continue
    else:
        # NOTE: Needs modification if applied by timesteps rather than annual potential.\
        total_energy += (n.generators_t.p_set.loc[:,ig] * n.links.loc[discharge_link,:].efficiency).sum()

    # For next cascade (This will be the one that is attached to the generator)
    next_water_bus = n.links.loc[discharge_link,:].bus2.split()[0] + " Water Bus"
    while True:
        
        if STC_check(next_water_bus):
            break
        
        if sum(n.links.bus0 == next_water_bus) == 0: # Hit terminal asset in the cascade when there are not more links starting in the reservoir
            break
    
        next_discharge_link = n.links[n.links.bus0 == next_water_bus].index[0].split()[0] + " Discharge Link"
        total_energy += (n.generators_t.p_set.loc[:,ig] * n.links.loc[next_discharge_link,:].efficiency).sum()
        next_water_bus = n.links.loc[next_discharge_link,:].bus2.split()[0] + " Water Bus" # -> BC_ALH_RES


In [286]:
100 * (total_pv) / total_load

0.0024852234197214914

In [ ]:
calculate_vre_energy(n.generp_max_pu, p_nom)

In [248]:
n.generators

,bus,control,p_nom,carrier,marginal_cost,p_nom_opt,type,p_nom_extendable,p_nom_min,p_nom_max,...,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
Generator,,,,,,,,,,,,,,,,,,,,,
BC_ABN_GSS RoR Generator,EastKootenay,Slack,24.00,,1.860000,24.00,,False,0.0,inf,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BC_ALK_DSS RoR Generator,Columbia-Shuswap,PQ,10.00,,2.190000,10.00,,False,0.0,inf,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BC_ASH_GSS RoR Generator,Alberni-Clayoquot,PQ,28.00,,1.860000,28.00,,False,0.0,inf,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BC_ASL_GSS RoR Generator,Squamish-Lillooet,PQ,50.00,,2.190000,50.00,,False,0.0,inf,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BC_BBR_DSS RoR Generator,FraserValley,PQ,6.00,,2.190000,6.00,,False,0.0,inf,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BC_CSS_GSS Wind Generator,MountWaddington,PQ,99.00,,0.000001,99.00,,False,0.0,inf,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BC_MKL_GSS Wind Generator,PeaceRiver,PQ,72.28,,0.000001,72.28,,False,0.0,inf,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BC_PSW_GSS Wind Generator,Thompson-Nicola,PQ,16.00,,0.000001,16.00,,False,0.0,inf,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0


### Hourly Energy check
i) Using a priority order of dispatch

In [ ]:
def calculate_vre_energy(p_max_pu, p_nom):
    total = 0
    for name,p in p_nom.items():
        total += sum(p_max_pu[name] * p)

    return total

In [30]:
# Get VRE time series
vre_cols = [col for col in n.generators.index if 'RoR Generator' in col] + [col for col in n.generators.index if 'Wind Generator' in col] + [col for col in n.generators.index if 'Solar Generator' in col]
for idx,col in enumerate(vre_cols):
    if idx == 0:
        vre = n.generators_t.p_max_pu.loc[:,col] * n.generators.loc[col,"p_nom"]
    else:
        vre += n.generators_t.p_max_pu.loc[:,col] * n.generators.loc[col,"p_nom"]
# total_ror = calculate_vre_energy(n.generators_t.p_max_pu.loc[:,ror_cols], n.generators.loc[ror_cols,:].p_nom)

# Get Net Load (NL) timeseries
nl = n.loads_t.p_set.sum(axis=1) - vre

In [39]:
inflow_cols = [col for col in n.generators.index if 'Inflow Generator' in col] 

In [60]:
n.generators_t.p_set.head(1) #.loc[:,inflow_cols]

Generator,BC_ALH_RES Inflow Generator,BC_ALU_RES Inflow Generator,BC_BR0_RES Inflow Generator,BC_GMS_RES Inflow Generator,BC_LAJ_RES Inflow Generator,BC_LDR_RES Inflow Generator,BC_MCA_RES Inflow Generator,BC_REV_RES Inflow Generator,BC_SCA_RES Inflow Generator,BC_SEV_RES Inflow Generator,BC_SON_RES Inflow Generator,BC_WGS_RES Inflow Generator,BC_WDN_RES Inflow Generator,BC_WHN_RES Inflow Generator
snapshot,,,,,,,,,,,,,,
2021-01-01,588194.606802,44569.983017,80578.649604,935526.195395,20300.662309,28915.105811,452748.145719,191175.356451,164014.194437,1.663948e+06,20792.853533,8038.150609,4.296536,1.469556


In [63]:
# create 2D list of reservoirs
# Rows are different cascades

# i) load in file with reservoir orders
cascade_df = pd.read_csv("/mnt/c/Users/pmcw9/Delta-E/PICS/Data/custom/hydro_res_wup_features.csv")





# inflow_generators = [ig for ig in n.generators.index if 'Inflow Generator' in ig] #.remove(['BC_WDN_RES Inflow Generator','BC_WHN_RES Inflow Generator'])
# inflow_generators.remove('BC_WDN_RES Inflow Generator') # To be added later (ror-water)
# inflow_generators.remove('BC_WHN_RES Inflow Generator') # To be added later (ror-water)

for _,row in cascade_df.iterrows():
    if row["asset_id"] == 
    if row["cascade_order"] == 1:




In [64]:
cascade_df

,asset_id,latitude,longitude,min_storage,max_storage,min_level,max_level,cascade_group,cascade_order
0,BC_GMS_RES,56.014797,-122.195744,0,3.947100e+10,624.030,672.08,Peace,1
1,BC_PCN_RES,55.982571,-121.994970,0,2.469000e+07,500.000,502.92,Peace,2
2,BC_STC_RES,56.196554,-120.913957,2145000000,2.310000e+09,460.011,461.80,Peace,3
3,BC_MCA_RES,52.075872,-118.570515,0,1.480000e+10,707.410,754.68,Mica/Columbia,1
4,BC_REV_RES,51.048563,-118.193945,0,1.730000e+08,571.500,573.02,Mica/Columbia,2
5,BC_WHN_RES,50.813937,-118.060010,0,4.593500e+05,698.000,701.95,Mica/Columbia,2
6,BC_WGS_RES,49.910565,-118.071814,0,1.215000e+08,634.000,641.30,Mica/Columbia,2
7,BC_ALH_RES,49.343024,-117.768849,0,8.760000e+09,418.640,440.10,Mica/Columbia,3
8,BC_LAJ_RES,50.839720,-122.855890,0,7.056000e+08,707.670,749.81,Bridge,1
9,BC_BR0_RES,50.729850,-122.240038,0,1.012500e+09,606.550,651.08,Bridge,2


In [ ]:
# 1) Find an Inflow Generator
# Start with an Inflow Generator 
inflow_generators = [ig for ig in n.generators.index if 'Inflow Generator' in ig] #.remove(['BC_WDN_RES Inflow Generator','BC_WHN_RES Inflow Generator'])
inflow_generators.remove('BC_WDN_RES Inflow Generator') # To be added later (ror-water)
inflow_generators.remove('BC_WHN_RES Inflow Generator') # To be added later (ror-water)
total_energy = 0

for ig in inflow_generators: #inflow_generators:
    water_bus = ig.split()[0] + " Water Bus" # reservoir water bus
    
    if STC_check(water_bus): # Special case for STC since removal
        continue

    discharge_link = n.links[n.links.bus0 == water_bus].index[0].split()[0] + " Discharge Link" 
    
    if sum(n.generators_t.p_set.columns == ig) == 0:
        # There is no inflow generator
        # This will produce 0 energy for all assets downstream
        continue
    else:
        # NOTE: Needs modification if applied by timesteps rather than annual potential.\
        total_energy += (n.generators_t.p_set.loc[:,ig] * n.links.loc[discharge_link,:].efficiency).sum()

    # For next cascade (This will be the one that is attached to the generator)
    next_water_bus = n.links.loc[discharge_link,:].bus2.split()[0] + " Water Bus"
    while True:
        
        if STC_check(next_water_bus):
            break
        
        if sum(n.links.bus0 == next_water_bus) == 0: # Hit terminal asset in the cascade when there are not more links starting in the reservoir
            break
    
        next_discharge_link = n.links[n.links.bus0 == next_water_bus].index[0].split()[0] + " Discharge Link"
        total_energy += (n.generators_t.p_set.loc[:,ig] * n.links.loc[next_discharge_link,:].efficiency).sum()
        next_water_bus = n.links.loc[next_discharge_link,:].bus2.split()[0] + " Water Bus"

In [33]:
# Inflow missing for: ['BC_JHT_RES Inflow Generator', 'BC_PCN_RES Inflow Generator', 'BC_RUS_RES Inflow Generator', 'BC_SFL_RES Inflow Generator', 'BC_STC_RES Inflow Generator']
# Reservoir setup
# Need table of 
soc = pd.Series([0]*nl.shape[0],index=nl.index)




# Simulated dispatch
for timestamp,value in nl.items():
    print(timestamp, value)
    if value > 0:
        soc = inflow 
    else:
        soc = 
        pass
        

NameError: name 'storage' is not defined

In [36]:
pd.Series([0]*nl.shape[0],index=nl.index)

snapshot
2021-01-01 00:00:00    0
2021-01-01 01:00:00    0
2021-01-01 02:00:00    0
2021-01-01 03:00:00    0
2021-01-01 04:00:00    0
                      ..
2021-12-31 19:00:00    0
2021-12-31 20:00:00    0
2021-12-31 21:00:00    0
2021-12-31 22:00:00    0
2021-12-31 23:00:00    0
Length: 8760, dtype: int64